In [1]:
# | default_exp backend.tools.fs_read_backup

In [2]:
# | export
from agentic.tools.base import Tool
from pathlib import Path
from typing import List, Dict, Any
import fnmatch

In [3]:
# | export
class FsReadTool(Tool):
    def get_tool_schema(self) -> Dict[str, Any]:
        """Return the OpenAI-compatible schema for fs_read."""
        return {
            "type": "function",
            "function": {
                "name": "fs_read",
                "description": "File system operations: read files with intelligent chunking, list directories with depth, find files, and grep across files.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "operations": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "mode": {
                                        "type": "string",
                                        "enum": ["Line", "Directory", "Find", "Grep"],
                                        "description": "Operation mode: Line (read file), Directory (list with depth), Find (find files by name), Grep (search across files)"
                                    },
                                    "path": {"type": "string", "description": "File or directory path"},
                                    "pattern": {"type": "string", "description": "Search pattern for Grep mode"},
                                    "name_pattern": {"type": "string", "description": "File name pattern for Find mode"},
                                    "file_pattern": {"type": "string", "description": "File pattern for Grep mode (e.g., '*.py')"},
                                    "depth": {"type": "integer", "description": "Directory depth for Directory mode"},
                                    "max_depth": {"type": "integer", "description": "Maximum depth for Find mode"},
                                    "context_lines": {"type": "integer", "description": "Context lines around matches"},
                                    "start_line": {"type": "integer", "description": "Start line for Line mode"},
                                    "end_line": {"type": "integer", "description": "End line for Line mode (-1 for end)"},
                                    "show_hidden": {"type": "boolean", "description": "Show hidden files/directories"},
                                    "case_sensitive": {"type": "boolean", "description": "Case sensitive search"},
                                    "recursive": {"type": "boolean", "description": "Recursive search for Grep mode"},
                                    "file_type": {"type": "string", "enum": ["file", "dir", "all"], "description": "File type filter for Find mode"},
                                    "smart_chunk": {"type": "boolean", "description": "Use intelligent chunking for large files"}
                                },
                                "required": ["mode", "path"]
                            }
                        }
                    },
                    "required": ["operations"]
                }
            }
        }

    def execute(self, operations: List[Dict[str, Any]]) -> Dict[str, Any]:
        """Execute fs_read with intelligent chunking and advanced features."""
        results = []
        valid_modes = {"directory", "line", "find", "grep"}
        
        for op in operations:
            mode = op.get("mode").lower()
            path = op.get("path")

            if mode not in valid_modes:
                results.append({
                    "path": path,
                    "error": f"Invalid mode '{mode}'. Valid modes: {', '.join(valid_modes)}"
                })
                continue
            
            try:
                if mode == "directory":
                    depth = op.get("depth", 1)
                    show_hidden = op.get("show_hidden", False)
                    items = self._list_directory_tree(path, depth, show_hidden)
                    results.append({"path": path, "items": items, "depth": depth})
                
                elif mode == "line":
                    start_line = op.get("start_line", 1)
                    end_line = op.get("end_line", -1)
                    smart_chunk = op.get("smart_chunk", True)
                    content = self._read_file_intelligent(path, start_line, end_line, smart_chunk)
                    results.append({"path": path, "content": content, "start_line": start_line, "end_line": end_line})
                
                elif mode == "find":
                    name_pattern = op.get("name_pattern", "*")
                    file_type = op.get("file_type", "all")
                    max_depth = op.get("max_depth", 10)
                    found_items = self._find_files(path, name_pattern, file_type, max_depth)
                    results.append({"path": path, "name_pattern": name_pattern, "found": found_items})
                
                elif mode == "grep":
                    pattern = op.get("pattern", "")
                    file_pattern = op.get("file_pattern", "*")
                    recursive = op.get("recursive", True)
                    context_lines = op.get("context_lines", 2)
                    matches = self._grep_files(path, pattern, file_pattern, recursive, context_lines)
                    results.append({"path": path, "pattern": pattern, "matches": matches})
            
            except Exception as e:
                results.append({"path": path, "error": str(e)})
        
        return {"results": results}

    def _get_file_info(self, path: str) -> Dict[str, Any]:
        """Get comprehensive file information for intelligent reading decisions."""
        path_obj = Path(path)
        if not path_obj.exists():
            return {"error": "File not found"}
        
        stat = path_obj.stat()
        file_size = stat.st_size
        
        file_info = {
            "size": file_size,
            "lines": 0,
            "is_binary": False,
            "encoding": "utf-8",
            "file_type": path_obj.suffix.lower(),
            "is_large": file_size > 1024 * 1024,  # > 1MB
            "estimated_lines": file_size // 50 if file_size > 0 else 0
        }
        
        try:
            with open(path, 'rb') as f:
                chunk = f.read(1024)
                if b'\x00' in chunk:
                    file_info["is_binary"] = True
                    return file_info
        except:
            file_info["is_binary"] = True
            return file_info
        
        try:
            if file_size < 10 * 1024 * 1024:  # < 10MB
                with open(path, 'r', encoding='utf-8', errors='ignore') as f:
                    file_info["lines"] = sum(1 for _ in f)
            else:
                with open(path, 'r', encoding='utf-8', errors='ignore') as f:
                    sample_lines = sum(1 for _, line in zip(range(10000), f))
                    file_info["lines"] = int((file_size / (f.tell() or 1)) * sample_lines)
        except:
            pass
        
        return file_info

    def _decide_read_strategy(self, path: str, start_line: int, end_line: int, file_info: Dict) -> Dict[str, Any]:
        """Decide how to read the file based on size, type, and request."""
        strategy = {
            "method": "full",
            "chunk_size": 1000,
            "sample_ratio": 0.1,
            "reason": ""
        }
        
        total_lines = file_info.get("lines", 0)
        file_size = file_info.get("size", 0)
        
        if file_info.get("is_binary"):
            strategy.update({"method": "binary_info", "reason": "Binary file detected"})
        elif file_size > 50 * 1024 * 1024:
            strategy.update({"method": "head_tail", "chunk_size": 500, "reason": "Very large file, showing head and tail"})
        elif total_lines > 10000 and (end_line - start_line + 1) > 5000:
            strategy.update({"method": "sampled", "sample_ratio": 0.2, "reason": "Large file with many requested lines"})
        elif total_lines > 5000 and (start_line == 1 and end_line == total_lines):
            strategy.update({"method": "chunked", "chunk_size": 2000, "reason": "Large file, full read requested"})
        elif (end_line - start_line + 1) > 2000:
            strategy.update({"method": "chunked", "chunk_size": 1000, "reason": "Many lines requested"})
        else:
            strategy.update({"method": "full", "reason": "Standard read"})
        
        return strategy

    def _read_file_intelligent(self, path: str, start_line: int = 1, end_line: int = -1, smart_chunk: bool = True) -> str:
        """Intelligently read file based on size and content type."""
        if not smart_chunk:
            return self._read_file_lines(path, start_line, end_line)
        
        file_info = self._get_file_info(path)
        if "error" in file_info:
            return file_info["error"]
        
        if file_info.get("is_binary"):
            return f"[Binary file: {file_info['size']} bytes, type: {file_info.get('file_type', 'unknown')}]"
        
        strategy = self._decide_read_strategy(path, start_line, end_line, file_info)
        
        try:
            with open(path, 'r', encoding='utf-8', errors='ignore') as f:
                lines = f.readlines()
            
            total_lines = len(lines)
            if end_line == -1:
                end_line = total_lines
            
            start_idx = max(0, start_line - 1)
            end_idx = min(total_lines, end_line)
            
            if strategy["method"] == "full":
                content = ''.join(lines[start_idx:end_idx])
            
            elif strategy["method"] == "chunked":
                chunk_size = strategy["chunk_size"]
                chunks = []
                current_idx = start_idx
                
                while current_idx < end_idx:
                    chunk_end = min(current_idx + chunk_size, end_idx)
                    chunk = ''.join(lines[current_idx:chunk_end])
                    chunks.append(f"--- Lines {current_idx + 1}-{chunk_end} ---\n{chunk}")
                    current_idx = chunk_end
                    
                    if len(chunks) >= 5:
                        remaining = end_idx - current_idx
                        if remaining > 0:
                            chunks.append(f"... [{remaining} more lines omitted] ...")
                        break
                
                content = '\n'.join(chunks)
            
            elif strategy["method"] == "head_tail":
                chunk_size = strategy["chunk_size"]
                head = ''.join(lines[start_idx:start_idx + chunk_size])
                tail = ''.join(lines[max(start_idx + chunk_size, end_idx - chunk_size):end_idx])
                
                omitted = end_idx - start_idx - (2 * chunk_size)
                content = f"--- Head (lines {start_idx + 1}-{start_idx + chunk_size}) ---\n{head}\n"
                if omitted > 0:
                    content += f"... [{omitted} lines omitted] ...\n"
                content += f"--- Tail (lines {end_idx - chunk_size + 1}-{end_idx}) ---\n{tail}"
            
            elif strategy["method"] == "sampled":
                sample_ratio = strategy["sample_ratio"]
                total_requested = end_idx - start_idx
                sample_size = int(total_requested * sample_ratio)
                
                if sample_size < 100:
                    sample_size = min(100, total_requested)
                
                step = max(1, total_requested // sample_size)
                sampled_lines = []
                
                for i in range(start_idx, end_idx, step):
                    if len(sampled_lines) >= sample_size:
                        break
                    sampled_lines.append(f"{i + 1:6d}: {lines[i]}")
                
                content = f"--- Sampled {len(sampled_lines)} lines from {total_requested} total ---\n"
                content += ''.join(sampled_lines)
            
            metadata = f"\n--- File Info: {file_info['size']} bytes, {file_info['lines']} lines, Strategy: {strategy['method']} ({strategy['reason']}) ---"
            return content + metadata
            
        except Exception as e:
            return f"Error reading file: {str(e)}"

    def _read_file_lines(self, path: str, start_line: int, end_line: int) -> str:
        """Simple line-based file reading."""
        with open(path, 'r', encoding='utf-8', errors='ignore') as f:
            lines = f.readlines()
        
        if end_line == -1:
            end_line = len(lines)
        
        start_idx = max(0, start_line - 1)
        end_idx = min(len(lines), end_line)
        
        return ''.join(lines[start_idx:end_idx])

    def _list_directory_tree(self, path: str, depth: int, show_hidden: bool) -> List[Dict]:
        """List directory with depth and metadata."""
        items = []
        path_obj = Path(path)
        
        if not path_obj.exists() or not path_obj.is_dir():
            return items
            
        def _scan_dir(dir_path: Path, current_depth: int) -> List[Dict]:
            dir_items = []
            if current_depth > depth:
                return dir_items
                
            try:
                for item in sorted(dir_path.iterdir()):
                    if not show_hidden and item.name.startswith('.'):
                        continue
                        
                    stat = item.stat()
                    item_info = {
                        "name": item.name,
                        "type": "directory" if item.is_dir() else "file",
                        "size": stat.st_size if item.is_file() else None,
                        "modified": stat.st_mtime,
                        "permissions": oct(stat.st_mode)[-3:],
                        "path": str(item)
                    }
                    
                    if item.is_dir() and current_depth < depth:
                        item_info["children"] = _scan_dir(item, current_depth + 1)
                    
                    dir_items.append(item_info)
            except PermissionError:
                pass
                
            return dir_items
        
        return _scan_dir(path_obj, 0)

    def _search_in_file(self, path: str, pattern: str, context_lines: int, case_sensitive: bool) -> List[Dict]:
        """Search pattern in file with context."""
        matches = []
        
        try:
            with open(path, 'r', encoding='utf-8', errors='ignore') as f:
                lines = f.readlines()
        except:
            return matches
        
        search_pattern = pattern if case_sensitive else pattern.lower()
        
        for i, line in enumerate(lines):
            search_line = line if case_sensitive else line.lower()
            
            if search_pattern in search_line:
                start_ctx = max(0, i - context_lines)
                end_ctx = min(len(lines), i + context_lines + 1)
                
                context = []
                for j in range(start_ctx, end_ctx):
                    context.append({
                        "line_number": j + 1,
                        "content": lines[j].rstrip(),
                        "is_match": j == i
                    })
                
                matches.append({
                    "line_number": i + 1,
                    "content": line.rstrip(),
                    "context": context
                })
        
        return matches

    def _find_files(self, path: str, name_pattern: str, file_type: str, max_depth: int) -> List[Dict]:
        """Find files by name pattern."""
        found = []
        path_obj = Path(path)
        
        def _search_recursive(dir_path: Path, current_depth: int):
            if current_depth > max_depth:
                return
                
            try:
                for item in dir_path.iterdir():
                    if fnmatch.fnmatch(item.name, name_pattern):
                        if (file_type == "all" or 
                            (file_type == "file" and item.is_file()) or
                            (file_type == "dir" and item.is_dir())):
                            
                            stat = item.stat()
                            found.append({
                                "name": item.name,
                                "path": str(item),
                                "type": "directory" if item.is_dir() else "file",
                                "size": stat.st_size if item.is_file() else None,
                                # "modified": stat.st_mtime  
                            })
                    
                    if item.is_dir() and current_depth < max_depth:
                        _search_recursive(item, current_depth + 1)
            except PermissionError:
                pass
        
        if path_obj.is_dir():
            _search_recursive(path_obj, 0)
        
        return found

    def _grep_files(self, path: str, pattern: str, file_pattern: str, recursive: bool, context_lines: int) -> List[Dict]:
        """Grep pattern across multiple files."""
        matches = []
        path_obj = Path(path)
        
        def _grep_file(file_path: Path):
            try:
                file_matches = self._search_in_file(str(file_path), pattern, context_lines, False)
                if file_matches:
                    matches.append({
                        "file": str(file_path),
                        "matches": file_matches
                    })
            except:
                pass
        
        def _scan_directory(dir_path: Path):
            try:
                for item in dir_path.iterdir():
                    if item.is_file() and fnmatch.fnmatch(item.name, file_pattern):
                        _grep_file(item)
                    elif item.is_dir() and recursive:
                        _scan_directory(item)
            except PermissionError:
                pass
        
        if path_obj.is_file():
            _grep_file(path_obj)
        elif path_obj.is_dir():
            _scan_directory(path_obj)
        
        return matches

    def _generate_tree(self, path: str, max_depth: int, show_hidden: bool) -> str:
        """Generate tree-like directory structure."""
        tree_lines = []
        path_obj = Path(path)
        
        def _build_tree(dir_path: Path, prefix: str, current_depth: int):
            if current_depth > max_depth:
                return
                
            try:
                items = sorted(dir_path.iterdir(), key=lambda x: (x.is_file(), x.name.lower()))
                if not show_hidden:
                    items = [item for item in items if not item.name.startswith('.')]
                
                for i, item in enumerate(items):
                    is_last = i == len(items) - 1
                    current_prefix = "└── " if is_last else "├── "
                    tree_lines.append(f"{prefix}{current_prefix}{item.name}")
                    
                    if item.is_dir() and current_depth < max_depth:
                        next_prefix = prefix + ("    " if is_last else "│   ")
                        _build_tree(item, next_prefix, current_depth + 1)
            except PermissionError:
                tree_lines.append(f"{prefix}└── [Permission Denied]")
        
        tree_lines.append(str(path_obj))
        if path_obj.is_dir():
            _build_tree(path_obj, "", 0)
        
        return "\n".join(tree_lines)

In [4]:
## Example usage 

In [6]:
fs = FsReadTool()
fs._list_directory_tree("../",2,True)

[{'name': '.ipynb_checkpoints',
  'type': 'directory',
  'size': None,
  'modified': 1758054614.2661624,
  'permissions': '775',
  'path': '../.ipynb_checkpoints',
  'children': [{'name': 'analyzer-checkpoint.ipynb',
    'type': 'file',
    'size': 33594,
    'modified': 1758055336.882689,
    'permissions': '664',
    'path': '../.ipynb_checkpoints/analyzer-checkpoint.ipynb'},
   {'name': 'llm_factory-checkpoint.ipynb',
    'type': 'file',
    'size': 104749,
    'modified': 1758050950.6899283,
    'permissions': '664',
    'path': '../.ipynb_checkpoints/llm_factory-checkpoint.ipynb'},
   {'name': 'schemas-checkpoint.ipynb',
    'type': 'file',
    'size': 13262,
    'modified': 1758110822.1125474,
    'permissions': '664',
    'path': '../.ipynb_checkpoints/schemas-checkpoint.ipynb'},
   {'name': 'tokens-checkpoint.ipynb',
    'type': 'file',
    'size': 72,
    'modified': 1757871031.1988478,
    'permissions': '664',
    'path': '../.ipynb_checkpoints/tokens-checkpoint.ipynb'}]},
 

In [2]:
# | export 
import os
import tempfile
import pytest
from pathlib import Path



@pytest.fixture
def sample_dir_structure():
    """Create a temporary directory with files for testing."""
    with tempfile.TemporaryDirectory() as tmpdir:
        root = Path(tmpdir)
        # Create some files
        (root / "file1.txt").write_text("hello world\nfoo bar\nbaz qux\n")
        (root / "file2.log").write_text("log line 1\nlog line 2\n")
        # Create subdirectory
        subdir = root / "subdir"
        subdir.mkdir()
        (subdir / "nested.txt").write_text("nested content\nanother line\n")
        yield root


def test_directory_mode_lists_files(sample_dir_structure):
    tool = FsReadTool()
    ops = [{"mode": "Directory", "path": str(sample_dir_structure), "depth": 2}]
    result = tool.execute(ops)
    assert "results" in result
    items = result["results"][0]["items"]
    assert any("file1.txt" in str(i) for i in items)
    assert any("subdir" in str(i) for i in items)


def test_line_mode_reads_partial_content(sample_dir_structure):
    tool = FsReadTool()
    file_path = sample_dir_structure / "file1.txt"
    ops = [{"mode": "Line", "path": str(file_path), "start_line": 2, "end_line": 3}]
    result = tool.execute(ops)
    content = result["results"][0]["content"]
    assert "foo bar" in content
    assert "hello world" not in content  # because we started from line 2


def test_find_mode_finds_files(sample_dir_structure):
    tool = FsReadTool()
    ops = [{"mode": "Find", "path": str(sample_dir_structure), "name_pattern": "*.txt"}]
    result = tool.execute(ops)
    found = result["results"][0]["found"]
    assert any("file1.txt" in str(f) for f in found)
    assert any("nested.txt" in str(f) for f in found)


def test_grep_mode_finds_pattern(sample_dir_structure):
    tool = FsReadTool()
    ops = [{"mode": "Grep", "path": str(sample_dir_structure), "pattern": "log line", "file_pattern": "*.log"}]
    result = tool.execute(ops)
    matches = result["results"][0]["matches"]
    assert any("log line 1" in m["line"] for m in matches)
    assert all(m["file"].endswith(".log") for m in matches)


def test_invalid_mode_returns_error(sample_dir_structure):
    tool = FsReadTool()
    ops = [{"mode": "InvalidMode", "path": str(sample_dir_structure)}]
    result = tool.execute(ops)
    assert "error" in result["results"][0]
    assert "Invalid mode" in result["results"][0]["error"]


ModuleNotFoundError: No module named 'pytest'